In [1]:
from tomoSegmentPipeline.utils.common import read_array, write_array
from tomoSegmentPipeline.utils import setup
from tomoSegmentPipeline.dataloader import tomoSegment_dataset
from cryoS2Sdrop.analyze import plot_centralSlices
from tomoSegmentPipeline.model import DeepFinder_model
from tomoSegmentPipeline.losses import Tversky_loss

import numpy as np
import matplotlib.pyplot as plt
import torch
import os
from torch.utils.data import Dataset, DataLoader

PARENT_PATH = setup.PARENT_PATH

%matplotlib inline
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [2]:
path_data, path_target = setup.get_paths(['tomo02', 'tomo04'], 'rawCET')

Ncl = 2
dim_in = 84
lr = 1e-4
weight_decay = 0
Lrnd = 0
augment_data = False

my_dataset = tomoSegment_dataset(path_data, path_target, dim_in, Ncl, Lrnd, augment_data)
len(my_dataset)

152

In [3]:
batch_size = 2
dloader = DataLoader(my_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [4]:
for batch in dloader:
    subtomo, target, mask, gt_subtomo = batch
    print(subtomo.shape, target.shape)
    break

!!!!!
!!!!!


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
Ncl = 2
dim_in = 84
lr = 1e-4
weight_decay = 0
Lrnd = 18
augment_data = True
batch_size = 22
pretrain_type = None

model = DeepFinder_model(Ncl, loss_fn, lr, weight_decay, pretrain_type)